In [1]:
import pandas 
import numpy as np
import os
import re
import operator
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from data import *
from constants import *
from models import *
from solver import *

In [2]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [3]:
root = "/mnt/raid/davech2y/3d_captioning/ShapeNetCore_vol/nrrd_256_filter_div_128_solid/"
captions = pandas.read_csv("captions.tablechair.csv").iloc[:600]
visual_contexts = np.load("data/visual_context.npy")
# captions = pandas.read_csv("captions.tablechair_prep.csv")

In [4]:
captions = Caption(captions)
captions.preprocess()
captions.tranform()
captions.sanity_check()

True

In [5]:
train_captions = captions.tranformed_csv.iloc[:500]
valid_captions = captions.tranformed_csv.iloc[500:600].reset_index(drop=True)
train_context = visual_contexts[:500]
valid_context = visual_contexts[500:600]

In [6]:
# shape_tranform = transforms.Compose([transforms.Resize(IMAGE_SIZE), transforms.ToTensor()])
# shape_train_ds = ShapeDataset(root, train_captions, shape_tranform)
# shape_train_dl = DataLoader(shape_train_ds, batch_size=10)
# shape_valid_ds = ShapeDataset(root, valid_captions, shape_tranform)
# shape_valid_dl = DataLoader(shape_valid_ds, batch_size=10)
# shape_dl = {
#     'train': shape_train_dl,
#     'valid': shape_valid_dl
# }

In [7]:
# model = Encoder().cuda()
# optimizer = optim.SGD(model.parameters(), lr=0.01)
# criterion = nn.CrossEntropyLoss()
# encoder_solver = EncoderSolver(optimizer, criterion)

In [8]:
# encoder_solver.train(model, shape_dl, 10)

In [9]:
input_size = captions.dict_word2idx.__len__() + 1
hidden_size = 512
decoder = Decoder(input_size, hidden_size).cuda()

In [10]:
caption_train_ds = CaptionDataset(train_context, train_captions.description.values.tolist())
caption_train_dl = DataLoader(caption_train_ds, batch_size=50, shuffle=False)
caption_valid_ds = CaptionDataset(valid_context, valid_captions.description.values.tolist())
caption_valid_dl = DataLoader(caption_valid_ds, batch_size=50, shuffle=False)
caption_dl = {
    "train": caption_train_dl,
    "valid": caption_valid_dl
}

In [11]:
decoder_solver = DecoderSolver(optim.RMSprop(decoder.parameters(), lr=0.001), nn.CrossEntropyLoss())

In [12]:
decoder_solver.train(decoder, caption_dl, 10000, 1000)

[epoch 1000/10000] train_loss: 5.649673, valid_loss: 6.253466
[epoch 2000/10000] train_loss: 5.646488, valid_loss: 6.273619
[epoch 3000/10000] train_loss: 5.644440, valid_loss: 6.276175
[epoch 4000/10000] train_loss: 5.652386, valid_loss: 6.298085
[epoch 5000/10000] train_loss: 5.642811, valid_loss: 6.302113
[epoch 6000/10000] train_loss: 5.642381, valid_loss: 6.302835
[epoch 7000/10000] train_loss: 5.641885, valid_loss: 6.298961
[epoch 8000/10000] train_loss: 5.642154, valid_loss: 6.280963
[epoch 9000/10000] train_loss: 5.642382, valid_loss: 6.283738
[epoch 10000/10000] train_loss: 5.641955, valid_loss: 6.298563
